In [1]:
import librosa

def detect_silence(audio_file, threshold):
    # 读取音频文件
    audio, sr = librosa.load(audio_file)
    
    # 计算音频的短时能量
    energy = librosa.feature.rms(audio, frame_length=2048, hop_length=512)[0]
    
    # 根据能量阈值检测静默段的位置
    silence_segments = []
    is_silence = True
    start = 0
    
    for i in range(len(energy)):
        if is_silence and energy[i] > threshold:
            is_silence = False
            start = librosa.frames_to_time(i, sr=sr)
        elif not is_silence and energy[i] <= threshold:
            is_silence = True
            end = librosa.frames_to_time(i, sr=sr)
            silence_segments.append((start, end))
    
    return silence_segments

# 示例用法
audio_file = '/home/xinying/Speaker_L/TIMIT_Dataset/data/cancat2/2-FAEM0_FRAM1=SA1.WAV'
threshold = 0.01  # 设置音量阈值

segments = detect_silence(audio_file, threshold)
for segment in segments:
    print(f"Silence segment: {segment[0]} - {segment[1]} seconds")

Silence segment: 0.25541950113378686 - 0.5108390022675737 seconds
Silence segment: 0.5572789115646258 - 0.6501587301587302 seconds
Silence segment: 0.7198185941043084 - 0.9287981859410431 seconds
Silence segment: 1.1145578231292517 - 1.253877551020408 seconds
Silence segment: 1.6021768707482993 - 1.7647165532879818 seconds
Silence segment: 1.8343764172335602 - 1.9272562358276644 seconds
Silence segment: 1.9504761904761905 - 2.159455782312925 seconds
Silence segment: 2.2987755102040817 - 2.6238548752834467 seconds
Silence segment: 2.647074829931973 - 2.8792743764172335 seconds
Silence segment: 3.01859410430839 - 3.1811337868480725 seconds
Silence segment: 7.244625850340136 - 7.383945578231293 seconds
Silence segment: 7.476825396825397 - 7.87156462585034 seconds
Silence segment: 7.894784580498866 - 8.010884353741497 seconds
Silence segment: 8.080544217687075 - 8.382403628117913 seconds
Silence segment: 8.52172335600907 - 8.614603174603175 seconds
Silence segment: 8.661043083900227 - 8.70

/tmp/ipykernel_3503106/4050820983.py:8: FutureWarning: Pass y=[ 0.         -0.00021362 -0.00018311 ...  0.00015259  0.00021362
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  energy = librosa.feature.rms(audio, frame_length=2048, hop_length=512)[0]


In [9]:
audio_file = '/home/xinying/Speaker_L/TIMIT_Dataset/data/cancat2/2-FAEM0_FRAM1=SA1.WAV'
threshold = 0.01  # 设置音量阈值
audio, sr = librosa.load(audio_file)
    
# 计算音频的短时能量
energy = librosa.feature.rms(audio, frame_length=2048, hop_length=512)[0]

# 根据能量阈值检测静默段的位置
silence_segments = []
is_silence = True
start = 0

for i in range(len(energy)):
    if is_silence and energy[i] > threshold:
        is_silence = False
        start = librosa.frames_to_time(i, sr=sr)
    elif not is_silence and energy[i] <= threshold:
        is_silence = True
        end = librosa.frames_to_time(i, sr=sr)
        silence_segments.append((start, end))
print(silence_segments)


[(0.25541950113378686, 0.5108390022675737), (0.5572789115646258, 0.6501587301587302), (0.7198185941043084, 0.9287981859410431), (1.1145578231292517, 1.253877551020408), (1.6021768707482993, 1.7647165532879818), (1.8343764172335602, 1.9272562358276644), (1.9504761904761905, 2.159455782312925), (2.2987755102040817, 2.6238548752834467), (2.647074829931973, 2.8792743764172335), (3.01859410430839, 3.1811337868480725), (7.244625850340136, 7.383945578231293), (7.476825396825397, 7.87156462585034), (7.894784580498866, 8.010884353741497), (8.080544217687075, 8.382403628117913), (8.52172335600907, 8.614603174603175), (8.661043083900227, 8.70748299319728), (8.777142857142858, 8.962902494331066), (9.032562358276644, 9.1718820861678), (9.311201814058958, 9.47374149659864), (9.659501133786849, 9.7059410430839)]


/tmp/ipykernel_3503106/1641505973.py:6: FutureWarning: Pass y=[ 0.         -0.00021362 -0.00018311 ...  0.00015259  0.00021362
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  energy = librosa.feature.rms(audio, frame_length=2048, hop_length=512)[0]


In [13]:
energy.shape

(440,)

In [34]:
import librosa
import soundfile as sf
import numpy as np

def detect_silence_intervals(audio_file, threshold, min_silence_duration):
    # 读取音频文件
    audio, sr = librosa.load(audio_file, sr=None)
    
    # 计算音频的短时能量
    energy = librosa.feature.rms(audio, frame_length=2048, hop_length=512)[0]
    
    # 根据能量阈值检测静音段的位置
    silence_intervals = []
    is_silence = False
    start = 0
    
    for i in range(len(energy)):
        if not is_silence and energy[i] <= threshold:
            is_silence = True
            start = librosa.frames_to_time(i, sr=sr)
        elif is_silence and energy[i] > threshold:
            is_silence = False
            end = librosa.frames_to_time(i, sr=sr)
            
            if end - start >= min_silence_duration:
                silence_intervals.append((start, end))
    
    return silence_intervals

def remove_silence_intervals(audio_file, intervals, output_file):
    # 读取原始音频文件
    audio, sr = librosa.load(audio_file, sr=None)
    
    # 创建一个布尔掩码，用于标记静音部分
    mask = np.ones_like(audio, dtype=bool)
    
    # 根据静音间隔标记静音部分
    for interval in intervals:
        # start_frame = librosa.time_to_frames(interval[0], sr=sr)
        # end_frame = librosa.time_to_frames(interval[1], sr=sr)
        start_frame = int(interval[0] * sr)
        end_frame = int(interval[1] * sr)
        print(start_frame, end_frame,sr)
        
        # 将静音部分的标记置为False
        mask[start_frame:end_frame] = False
    
    # 根据掩码移除静音部分
    new_audio = audio[mask]
    print(audio.shape, new_audio.shape)
    
    # 保存删除静音部分后的音频文件
    sf.write(output_file, new_audio, sr)

# 示例用法
audio_file = '/home/xinying/Speaker2/seperate_speech/output/output.wav'
threshold = 0.01  # 设置能量阈值
min_silence_duration = 1.0  # 设置最小的静音间隔持续时间
output_file = 'output/out.wav'  # 设置输出文件路径

silence_intervals = detect_silence_intervals(audio_file, threshold, min_silence_duration)
print(silence_intervals)
remove_silence_intervals(audio_file, silence_intervals, output_file)

[]
(301644,) (301644,)


/tmp/ipykernel_3503106/368935854.py:10: FutureWarning: Pass y=[0.         0.         0.         ... 0.01397705 0.03015137 0.01205444] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  energy = librosa.feature.rms(audio, frame_length=2048, hop_length=512)[0]


In [11]:
segments

[]